In [15]:
import pandas as pd
import numpy as np 
df = pd.read_excel('/Users/chase/Downloads/bpdata.xlsx')

In [16]:
df.head()

,date,y,t,g,trend,trend2,dum75q2,rrtaxu
0,1950-01-01,-428.265747,-1081.332153,-1117.348755,1,1,0,0.0
1,1950-04-01,-425.807556,-1072.665527,-1116.824219,2,4,0,0.0
2,1950-07-01,-422.493561,-1062.218994,-1121.543823,3,9,0,0.0
3,1950-10-01,-421.014496,-1055.743774,-1112.510742,4,16,0,0.0
4,1951-01-01,-420.063110,-1046.365845,-1094.527344,5,25,0,0.0


In [17]:
df.index = df.date
df.index = pd.to_datetime(df.index)

In [18]:
def gen_lags(df, variable, number_of_lags):
    lag = 1
    while lag < number_of_lags + 1:
        df[str(variable) + '-' +str(lag)] = df[str(variable)].shift(lag)
        lag +=1
    return df
        



In [19]:
variables = ['y', 't', 'g']

for variable in variables:
    gen_lags(df, variable, 4)

In [20]:
df.head()

,date,y,t,g,trend,trend2,dum75q2,rrtaxu,y-1,y-2,y-3,y-4,t-1,t-2,t-3,t-4,g-1,g-2,g-3,g-4
date,,,,,,,,,,,,,,,,,,,,
1950-01-01,1950-01-01,-428.265747,-1081.332153,-1117.348755,1,1,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1950-04-01,1950-04-01,-425.807556,-1072.665527,-1116.824219,2,4,0,0.0,-428.265747,NaN,NaN,NaN,-1081.332153,NaN,NaN,NaN,-1117.348755,NaN,NaN,NaN
1950-07-01,1950-07-01,-422.493561,-1062.218994,-1121.543823,3,9,0,0.0,-425.807556,-428.265747,NaN,NaN,-1072.665527,-1081.332153,NaN,NaN,-1116.824219,-1117.348755,NaN,NaN
1950-10-01,1950-10-01,-421.014496,-1055.743774,-1112.510742,4,16,0,0.0,-422.493561,-425.807556,-428.265747,NaN,-1062.218994,-1072.665527,-1081.332153,NaN,-1121.543823,-1116.824219,-1117.348755,NaN
1951-01-01,1951-01-01,-420.063110,-1046.365845,-1094.527344,5,25,0,0.0,-421.014496,-422.493561,-425.807556,-428.265747,-1055.743774,-1062.218994,-1072.665527,-1081.332153,-1112.510742,-1121.543823,-1116.824219,-1117.348755


In [21]:
df = df.dropna(axis = 0)

In [22]:
import statsmodels.api as sm # for OLS

X = df[['y-1', 'y-2', 'y-3','y-4', 'g-1', 'g-2', 'g-3','g-4','t-1', 't-2', 't-3','t-4']]
X = sm.add_constant(X)
estimates = {}
for variable in variables:
    estimates[str(variable)] = sm.OLS(df[str(variable)],X).fit()
    df['e_' + str(variable)] =  estimates[str(variable)].resid

Here I construct the residuals from the timing assumption of the VAR 

In [23]:
df['res'] = df['e_t'] - 2.08* df['e_g']
X = df[['e_g', 'e_t']]
X = sm.add_constant(X)
reg1 = sm.OLS(df['e_y'] ,X).fit()
reg1predict = reg1.fittedvalues
reg1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    e_y   R-squared:                       0.240
Model:                            OLS   Adj. R-squared:                  0.234
Method:                 Least Squares   F-statistic:                     35.58
Date:                Fri, 28 Feb 2020   Prob (F-statistic):           3.76e-14
Time:                        22:29:46   Log-Likelihood:                -247.21
No. Observations:                 228   AIC:                             500.4
Df Residuals:                     225   BIC:                             510.7
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -4.911e-12      0.048  -1.03e-10      1.000      -0.094       0.094
e_g            0.0653      0.023      2.853      0.005       0.020       0.110
e_t            0.1387      0.018      7.699      0.000       0.103       0.174
==============================================================================
Omnibus:                       10.188   Durbin-Watson:                   2.046
Prob(Omnibus):                  0.006   Jarque-Bera (JB):               21.296
Skew:                           0.057   Prob(JB):                     2.38e-05
Kurtosis:                       4.493   Cond. No.                         2.67
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [24]:
df['res'] = df['e_t'] - 2.08* df['e_g']
X = df[['e_g', 'res']]
X = sm.add_constant(X)
firststage1 = sm.OLS(df['e_t'] ,X).fit()
df['fs1'] = firststage1.fittedvalues
firststage1.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    e_t   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.620e+32
Date:                Fri, 28 Feb 2020   Prob (F-statistic):               0.00
Time:                        22:29:46   Log-Likelihood:                 7370.1
No. Observations:                 228   AIC:                        -1.473e+04
Df Residuals:                     225   BIC:                        -1.472e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -9.714e-17   1.48e-16     -0.658      0.511   -3.88e-16    1.94e-16
e_g            2.0800   1.31e-16   1.59e+16      0.000       2.080       2.080
res            1.0000   5.57e-17   1.79e+16      0.000       1.000       1.000
==============================================================================
Omnibus:                       14.467   Durbin-Watson:                   2.047
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               40.245
Skew:                           0.003   Prob(JB):                     1.82e-09
Kurtosis:                       5.058   Cond. No.                         5.24
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [25]:
## looks like by the f-stat it is a strong instrument

X = df[['fs1','e_g']]
X = sm.add_constant(X)
secondstage1 = sm.OLS(df['e_y'] ,X).fit()
secondstage1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    e_y   R-squared:                       0.240
Model:                            OLS   Adj. R-squared:                  0.234
Method:                 Least Squares   F-statistic:                     35.58
Date:                Fri, 28 Feb 2020   Prob (F-statistic):           3.76e-14
Time:                        22:29:46   Log-Likelihood:                -247.21
No. Observations:                 228   AIC:                             500.4
Df Residuals:                     225   BIC:                             510.7
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -4.911e-12      0.048  -1.03e-10      1.000      -0.094       0.094
fs1            0.1387      0.018      7.699      0.000       0.103       0.174
e_g            0.0653      0.023      2.853      0.005       0.020       0.110
==============================================================================
Omnibus:                       10.188   Durbin-Watson:                   2.046
Prob(Omnibus):                  0.006   Jarque-Bera (JB):               21.296
Skew:                           0.057   Prob(JB):                     2.38e-05
Kurtosis:                       4.493   Cond. No.                         2.67
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [26]:

X = df[['g','t','y-1', 'y-2', 'y-3','y-4', 'g-1', 'g-2', 'g-3','g-4','t-1', 't-2', 't-3','t-4']]
X = sm.add_constant(X)
reg2 = sm.OLS(df['y'],X).fit()
reg2.summary()
## Note how this is the same as the residual regression for the point estimates on t and g. 


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 3.851e+04
Date:                Fri, 28 Feb 2020   Prob (F-statistic):               0.00
Time:                        22:29:46   Log-Likelihood:                -247.21
No. Observations:                 228   AIC:                             524.4
Df Residuals:                     213   BIC:                             575.9
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.6188     11.204     -0.323      0.747     -25.704      18.466
g              0.0653      0.024      2.776      0.006       0.019       0.112
t              0.1387      0.019      7.491      0.000       0.102       0.175
y-1            1.1515      0.070     16.487      0.000       1.014       1.289
y-2           -0.1397      0.103     -1.352      0.178      -0.343       0.064
y-3           -0.0101      0.102     -0.099      0.921      -0.211       0.191
y-4            0.0067      0.069      0.096      0.923      -0.130       0.143
g-1           -0.0949      0.037     -2.593      0.010      -0.167      -0.023
g-2            0.0779      0.036      2.180      0.030       0.007       0.148
g-3           -0.0892      0.035     -2.538      0.012      -0.159      -0.020
g-4            0.0430      0.022      1.978      0.049       0.000       0.086
t-1           -0.1034      0.024     -4.249      0.000      -0.151      -0.055
t-2           -0.0444      0.025     -1.764      0.079      -0.094       0.005
t-3           -0.0381      0.025     -1.522      0.130      -0.087       0.011
t-4            0.0380      0.020      1.867      0.063      -0.002       0.078
==============================================================================
Omnibus:                       10.188   Durbin-Watson:                   2.046
Prob(Omnibus):                  0.006   Jarque-Bera (JB):               21.296
Skew:                           0.057   Prob(JB):                     2.38e-05
Kurtosis:                       4.493   Cond. No.                     7.58e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.58e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [27]:
X = df[['g','res','y-1', 'y-2', 'y-3','y-4', 'g-1', 'g-2', 'g-3','g-4','t-1', 't-2', 't-3','t-4']]
X = sm.add_constant(X)
firststage2 = sm.OLS(df['t'] ,X).fit()
df['fs2'] = firststage2.fittedvalues
firststage2.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      t   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.313e+25
Date:                Fri, 28 Feb 2020   Prob (F-statistic):               0.00
Time:                        22:29:46   Log-Likelihood:                 5129.6
No. Observations:                 228   AIC:                        -1.023e+04
Df Residuals:                     213   BIC:                        -1.018e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        132.1302   6.91e-10   1.91e+11      0.000     132.130     132.130
g              2.0800    2.5e-12   8.33e+11      0.000       2.080       2.080
res            1.0000   1.06e-12   9.42e+11      0.000       1.000       1.000
y-1            1.5037   3.86e-12    3.9e+11      0.000       1.504       1.504
y-2           -1.1029   5.89e-12  -1.87e+11      0.000      -1.103      -1.103
y-3            0.2089   5.86e-12   3.57e+10      0.000       0.209       0.209
y-4           -0.6155      4e-12  -1.54e+11      0.000      -0.615      -0.615
g-1           -2.6169   3.27e-12  -8.01e+11      0.000      -2.617      -2.617
g-2            0.3022   2.07e-12   1.46e+11      0.000       0.302       0.302
g-3            0.1660      2e-12    8.3e+10      0.000       0.166       0.166
g-4            0.2220    1.3e-12   1.71e+11      0.000       0.222       0.222
t-1            0.5255   1.19e-12   4.41e+11      0.000       0.526       0.526
t-2            0.2457   1.43e-12   1.72e+11      0.000       0.246       0.246
t-3           -0.0429   1.43e-12  -2.99e+10      0.000      -0.043      -0.043
t-4            0.2458   1.19e-12   2.06e+11      0.000       0.246       0.246
==============================================================================
Omnibus:                        7.843   Durbin-Watson:                   0.000
Prob(Omnibus):                  0.020   Jarque-Bera (JB):                4.192
Skew:                           0.088   Prob(JB):                        0.123
Kurtosis:                       2.360   Cond. No.                     7.77e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.77e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [28]:
X = df[['fs2','g','y-1', 'y-2', 'y-3','y-4', 'g-1', 'g-2', 'g-3','g-4','t-1', 't-2', 't-3','t-4']]
X = sm.add_constant(X)
secondstage2 = sm.OLS(df['y'],X).fit()
secondstage2.summary()

## Note they both match

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 3.851e+04
Date:                Fri, 28 Feb 2020   Prob (F-statistic):               0.00
Time:                        22:29:46   Log-Likelihood:                -247.21
No. Observations:                 228   AIC:                             524.4
Df Residuals:                     213   BIC:                             575.9
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.6188     11.204     -0.323      0.747     -25.704      18.466
fs2            0.1387      0.019      7.491      0.000       0.102       0.175
g              0.0653      0.024      2.776      0.006       0.019       0.112
y-1            1.1515      0.070     16.487      0.000       1.014       1.289
y-2           -0.1397      0.103     -1.352      0.178      -0.343       0.064
y-3           -0.0101      0.102     -0.099      0.921      -0.211       0.191
y-4            0.0067      0.069      0.096      0.923      -0.130       0.143
g-1           -0.0949      0.037     -2.593      0.010      -0.167      -0.023
g-2            0.0779      0.036      2.180      0.030       0.007       0.148
g-3           -0.0892      0.035     -2.538      0.012      -0.159      -0.020
g-4            0.0430      0.022      1.978      0.049       0.000       0.086
t-1           -0.1034      0.024     -4.249      0.000      -0.151      -0.055
t-2           -0.0444      0.025     -1.764      0.079      -0.094       0.005
t-3           -0.0381      0.025     -1.522      0.130      -0.087       0.011
t-4            0.0380      0.020      1.867      0.063      -0.002       0.078
==============================================================================
Omnibus:                       10.188   Durbin-Watson:                   2.046
Prob(Omnibus):                  0.006   Jarque-Bera (JB):               21.296
Skew:                           0.057   Prob(JB):                     2.38e-05
Kurtosis:                       4.493   Cond. No.                     7.58e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.58e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""